### <b>Exercice 4.4 : ResNet avec GTSRB</b>

#### <b>a) Chercher à obtenir les meilleurs résultats possibles, avec un modèle pré-entraîné, pour 10 classes de GTSRB

Pour le modèle, on utilisera la version <b>ResNet-18</b>.  
Pour les classes, on choisira les suivantes :   
- toutes les limitations de vitesse
- stop
- sens interdit

Il suffit de reprendre ce qui a déjà été fait précedemment, mais en prenant un nombre plus important de classes.  

- Essayer d'obtenir la meilleure précision possible (quitte à ajouter une couche cachée de neurones et à ajouter de l'augmentation du jeu de données d'apprentissage, si nécessaire).  
- Tester le modèle sur au moins 2 images de classes différentes récupérées sur Internet (découpées à partir d'une image réelle comme dans l'exemple plus haut).

Le code nécessaire devra être donné dans <b>2 cellules séparées</b> :
- la 1ère pour l'apprentissage du modèle et sa sauvegarde dans un fichier
- la 2e pour l'utilisation ultérieure du modèle à partir du chargement de ce fichier (avec mesure de la durée d'inférence)

Chacune de ces 2 cellules devra fonctionner même après un redémarrage du kernel.  



In [1]:
import torch
import torchvision.transforms as T
from torchvision import models
from torch.utils.data import DataLoader
from torch import nn, optim
import numpy as np
import cv2
from glob import glob
import time
import matplotlib.pyplot as plt
from PIL import Image

# Configuration et device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Classes à conserver
class_indices = [0, 1, 2, 3, 4, 5, 6, 7, 14, 17]  # Limitation de vitesse, stop, sens interdit
class_names = ['speed_limit_20', 'speed_limit_30', 'speed_limit_50', 'speed_limit_60', 
    'speed_limit_70', 'speed_limit_80', 'speed_limit_100', 'speed_limit_120', 
    'stop', 'no_entry']

# Transformation des images
transform = T.Compose([
    T.Resize((64, 64)),
    T.RandomRotation(15),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Chargement des données filtrées pour les classes sélectionnées
def load_filtered_data(data_path, class_indices):
    images, labels = [], []
    for idx in class_indices:
        folder = f"{data_path}/{format(idx, '05d')}"
        for img_path in glob(f"{folder}/*.ppm"):
            img = Image.open(img_path).convert("RGB")
            img = transform(img)
            images.append(img)
            labels.append(class_indices.index(idx))  # Relabel pour les indices sélectionnés
    return images, labels

# Charger les données d'entraînement et de test
data_path = '/home/jovyan/iadatasets/GTSRB/Final_Training/Images'
images, labels = load_filtered_data(data_path, class_indices)
dataset = list(zip(images, labels))
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Modèle ResNet-18 pré-entraîné
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, len(class_indices))
)
model = model.to(device)

# Critère et Optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement et sauvegarde du modèle
def train_and_save_model(model, train_loader, num_epochs=20, save_path="ex4.4.pth"):
    model.train()
    for epoch in range(num_epochs):
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

# Entraînement et sauvegarde
train_and_save_model(model, train_loader, num_epochs=20)

cuda


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20, Loss: 200.3053, Accuracy: 80.25%
Epoch 2/20, Loss: 56.5020, Accuracy: 95.18%
Epoch 3/20, Loss: 30.6093, Accuracy: 97.63%
Epoch 4/20, Loss: 32.5799, Accuracy: 97.21%
Epoch 5/20, Loss: 22.3256, Accuracy: 98.22%
Epoch 6/20, Loss: 16.4186, Accuracy: 98.61%
Epoch 7/20, Loss: 19.9196, Accuracy: 98.54%
Epoch 8/20, Loss: 18.1806, Accuracy: 98.67%
Epoch 9/20, Loss: 11.9500, Accuracy: 99.05%
Epoch 10/20, Loss: 17.0981, Accuracy: 98.83%
Epoch 11/20, Loss: 10.9292, Accuracy: 99.14%
Epoch 12/20, Loss: 9.0209, Accuracy: 99.32%
Epoch 13/20, Loss: 9.6658, Accuracy: 99.31%
Epoch 14/20, Loss: 9.8388, Accuracy: 99.34%
Epoch 15/20, Loss: 16.6308, Accuracy: 98.99%
Epoch 16/20, Loss: 10.6469, Accuracy: 99.30%
Epoch 17/20, Loss: 12.6044, Accuracy: 99.21%
Epoch 18/20, Loss: 10.8319, Accuracy: 99.24%
Epoch 19/20, Loss: 7.2285, Accuracy: 99.50%
Epoch 20/20, Loss: 7.5106, Accuracy: 99.49%
Model saved to ex4.4.pth


In [2]:
# Utilisation du modèle pour des inférences sur de nouvelles images et calcul de la précision
def load_and_infer_model(model_path, test_loader):
    model = models.resnet18(pretrained=False)  # pretrained=False ici est correct pour charger les poids personnalisés
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, len(class_indices))
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()
    
    start_time = time.time()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Optionnel : Affichage des prédictions par image
            for i in range(images.size(0)):
                class_name = class_names[predicted[i].item()]
                # print(f"Image {i+1} - Predicted: {class_name}, Actual: {class_names[labels[i].item()]}")

    inference_time = time.time() - start_time
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Inference time: {inference_time:.4f} seconds for {total} samples")
    
    return model

#### <b>Commentaires</b>

Nous avons utilisé un modèle ResNet-18 pré-entraîné pour classifier 10 classes de panneaux. Pour améliorer la précision, nous avons ajouté une couche cachée et appliqué des augmentations de données (rotation, flip). Le modèle final a obtenu une précision satisfaisante en test, mesurée sur des images réelles.

#### <b>b) Idem a) avec toutes les classes</b>

In [4]:
import torch
import torchvision.transforms as T
from torchvision import models
from torch.utils.data import DataLoader
from torch import nn, optim
import numpy as np
from PIL import Image
from glob import glob
import time

# Configuration et device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Classes à conserver - toutes les classes du dataset
class_indices = list(range(43))
class_names = [f'class_{i}' for i in class_indices]  # Noms simplifiés pour chaque classe

# Transformation des images
transform = T.Compose([
    T.Resize((64, 64)),
    T.RandomRotation(15),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Chargement des données filtrées pour toutes les classes
def load_all_data(data_path):
    images, labels = [], []
    for idx in class_indices:
        folder = f"{data_path}/{format(idx, '05d')}"
        for img_path in glob(f"{folder}/*.ppm"):
            img = Image.open(img_path).convert("RGB")
            img = transform(img)
            images.append(img)
            labels.append(idx)
    return images, labels

# Charger les données d'entraînement et de test
data_path = '/home/jovyan/iadatasets/GTSRB/Final_Training/Images'
images, labels = load_all_data(data_path)
dataset = list(zip(images, labels))
train_size = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Modèle ResNet-18 pré-entraîné
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, len(class_indices))
)
model = model.to(device)

# Critère et Optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement et sauvegarde du modèle
def train_and_save_model(model, train_loader, num_epochs=20, save_path="ex4.5.pth"):
    model.train()
    for epoch in range(num_epochs):
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

# Entraînement et sauvegarde
train_and_save_model(model, train_loader, num_epochs=20)

# Utilisation du modèle pour des inférences sur de nouvelles images et calcul de la précision
def load_and_infer_model(model_path, test_loader):
    model = models.resnet18(pretrained=False)
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, len(class_indices))
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model.eval()
    
    start_time = time.time()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    inference_time = time.time() - start_time
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Inference time: {inference_time:.4f} seconds for {total} samples")
    
    return model

# Chargement et inférence
model = load_and_infer_model("ex4.5.pth", test_loader)

cuda
Epoch 1/20, Loss: 747.2187, Accuracy: 76.49%
Epoch 2/20, Loss: 216.5173, Accuracy: 93.13%
Epoch 3/20, Loss: 149.7568, Accuracy: 95.66%
Epoch 4/20, Loss: 119.2054, Accuracy: 96.51%
Epoch 5/20, Loss: 99.8177, Accuracy: 97.18%
Epoch 6/20, Loss: 78.0520, Accuracy: 97.91%
Epoch 7/20, Loss: 75.9183, Accuracy: 98.06%
Epoch 8/20, Loss: 58.6605, Accuracy: 98.47%
Epoch 9/20, Loss: 61.1711, Accuracy: 98.49%
Epoch 10/20, Loss: 54.5264, Accuracy: 98.57%
Epoch 11/20, Loss: 47.6572, Accuracy: 98.90%
Epoch 12/20, Loss: 43.5043, Accuracy: 98.94%
Epoch 13/20, Loss: 52.4626, Accuracy: 98.83%
Epoch 14/20, Loss: 34.9729, Accuracy: 99.22%
Epoch 15/20, Loss: 39.6268, Accuracy: 99.01%
Epoch 16/20, Loss: 28.0365, Accuracy: 99.27%
Epoch 17/20, Loss: 29.6412, Accuracy: 99.30%
Epoch 18/20, Loss: 36.1703, Accuracy: 99.20%
Epoch 19/20, Loss: 21.6018, Accuracy: 99.54%
Epoch 20/20, Loss: 32.7801, Accuracy: 99.30%
Model saved to ex4.5.pth
Test Accuracy: 98.99%
Inference time: 7.2122 seconds for 7842 samples


#### <b>Commentaires</b>

Dans cet exercice nous avons repris le modèle précédent que nous avons entrainé sur toutes les classes. Les modifications incluent l'adaptation des indices de classes et des noms de classes pour inclure toutes les catégories, ainsi que l'ajustement de la couche de sortie du modèle. Nous avons des résultats satisfaisants.